# memstack-agent 完整示例

本 notebook 演示如何将所有组件组合在一起构建一个简单的 Agent。

## 概述

将之前学习的概念整合：
- 工具创建和注册
- 事件系统
- 上下文和配置
- 简单的执行循环

## 1. 导入所有必要模块

In [ ]:
import sys
sys.path.insert(0, '/Users/tiejunsun/github/agi-demos/src')

import asyncio
import json
import uuid
from datetime import datetime
from typing import Dict, List, Any, Optional, AsyncGenerator

# Core imports
from memstack_agent import (
    # Types
    AgentContext,
    EventType,
    function_to_tool,
    # Events
    AgentEvent,
    ThoughtEvent,
    ActEvent,
    ObserveEvent,
    TextStartEvent,
    TextDeltaEvent,
    TextEndEvent,
    CompleteEvent,
    ErrorEvent,
    # Tool
    ToolDefinition,
    ToolMetadata,
)
from memstack_agent.core.types import ProcessorConfig, ProcessorState, get_event_category

## 2. 定义工具集

In [ ]:
# 定义一组工具

@function_to_tool
async def search(query: str) -> str:
    """搜索互联网获取信息。
    
    Args:
        query: 搜索查询词
    """
    # 模拟搜索
    results = [
        {"title": f"Result {i+1} for '{query}'", "url": f"https://example.com/{i}"}
        for i in range(3)
    ]
    return json.dumps(results, ensure_ascii=False)


@function_to_tool
async def read_file(path: str) -> str:
    """读取文件内容。
    
    Args:
        path: 文件路径
    """
    # 模拟文件读取
    return f"Content of file: {path}\nThis is sample file content."


@function_to_tool
async def calculate(expression: str) -> str:
    """执行数学计算。
    
    Args:
        expression: 数学表达式
    """
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {e}"


# 工具注册表
tools: Dict[str, ToolDefinition] = {
    "search": search,
    "read_file": read_file,
    "calculate": calculate,
}

print("Registered tools:")
for name, tool in tools.items():
    print(f"  - {name}: {tool.description[:40]}...")

## 3. 简单 Agent 执行器

In [ ]:
class SimpleAgent:
    """简单的 Agent 执行器。"""
    
    def __init__(
        self,
        context: AgentContext,
        config: ProcessorConfig,
        tools: Dict[str, ToolDefinition],
    ):
        self.context = context
        self.config = config
        self.tools = tools
        self.state = ProcessorState.IDLE
        self._step = 0
    
    async def execute(
        self,
        user_message: str,
        tool_calls: Optional[List[Dict[str, Any]]] = None,
    ) -> AsyncGenerator[AgentEvent, None]:
        """执行 Agent 并生成事件流。
        
        Args:
            user_message: 用户消息
            tool_calls: 预定义的工具调用列表（用于模拟）
        """
        self.state = ProcessorState.THINKING
        self._step = 0
        
        try:
            # 1. 思考阶段
            yield ThoughtEvent(
                conversation_id=self.context.conversation_id,
                content=f"收到用户请求: {user_message}\n正在分析需要采取的行动...",
                step_index=self._step,
            )
            
            # 2. 如果有预定义的工具调用，执行它们
            if tool_calls:
                for call in tool_calls:
                    self._step += 1
                    
                    tool_name = call["name"]
                    tool_input = call["input"]
                    
                    if tool_name not in self.tools:
                        yield ErrorEvent(
                            conversation_id=self.context.conversation_id,
                            message=f"Unknown tool: {tool_name}",
                        )
                        continue
                    
                    # 发出 Act 事件
                    self.state = ProcessorState.ACTING
                    tool_exec_id = f"exec-{uuid.uuid4().hex[:8]}"
                    
                    yield ActEvent(
                        conversation_id=self.context.conversation_id,
                        tool_name=tool_name,
                        tool_input=tool_input,
                        tool_execution_id=tool_exec_id,
                        step_index=self._step,
                    )
                    
                    # 执行工具
                    self.state = ProcessorState.OBSERVING
                    import time
                    start = time.time()
                    
                    result = await self.tools[tool_name].execute(**tool_input)
                    duration = int((time.time() - start) * 1000)
                    
                    # 发出 Observe 事件
                    yield ObserveEvent(
                        conversation_id=self.context.conversation_id,
                        tool_name=tool_name,
                        result=result,
                        duration_ms=duration,
                        tool_execution_id=tool_exec_id,
                    )
                    
                    # 思考结果
                    self.state = ProcessorState.THINKING
                    yield ThoughtEvent(
                        conversation_id=self.context.conversation_id,
                        content=f"工具 {tool_name} 返回结果，继续处理...",
                        step_index=self._step,
                    )
            
            # 3. 生成最终答案
            yield TextStartEvent(conversation_id=self.context.conversation_id)
            
            answer = f"我已完成您的请求 '{user_message}'。"
            if tool_calls:
                answer += f" 使用了 {len(tool_calls)} 个工具调用。"
            
            # 模拟流式输出
            for word in answer.split():
                yield TextDeltaEvent(
                    conversation_id=self.context.conversation_id,
                    delta=word + " "
                )
                await asyncio.sleep(0.05)  # 模拟延迟
            
            yield TextEndEvent(
                conversation_id=self.context.conversation_id,
                full_text=answer
            )
            
            # 4. 完成
            self.state = ProcessorState.COMPLETED
            yield CompleteEvent(
                conversation_id=self.context.conversation_id,
                result=answer,
                tokens={"total": 500},
                cost=0.01,
            )
            
        except Exception as e:
            self.state = ProcessorState.ERROR
            yield ErrorEvent(
                conversation_id=self.context.conversation_id,
                message=str(e),
                details={"error_type": type(e).__name__}
            )

## 4. 创建会话并执行

In [ ]:
# 创建上下文和配置
context = AgentContext(
    session_id=f"session-{uuid.uuid4().hex[:8]}",
    conversation_id=f"conv-{uuid.uuid4().hex[:8]}",
    user_id="user-demo",
    project_id="project-notebook",
    model="claude-3-sonnet",
)

config = ProcessorConfig(
    model="claude-3-sonnet",
    temperature=0.0,
    max_steps=10,
)

print(f"Session: {context.session_id}")
print(f"Conversation: {context.conversation_id}")

In [ ]:
# 创建 Agent
agent = SimpleAgent(context, config, tools)

# 定义模拟的工具调用序列
tool_calls = [
    {"name": "search", "input": {"query": "Python asyncio tutorial"}},
    {"name": "calculate", "input": {"expression": "2 + 3 * 4"}},
]

# 执行并收集事件
print("=" * 60)
print("Agent Execution Flow")
print("=" * 60)

async def run_agent():
    events = []
    async for event in agent.execute("帮我搜索 Python asyncio 教程并计算 2+3*4", tool_calls):
        events.append(event)
        
        # 打印事件摘要
        event_type = event.event_type.value
        if isinstance(event, ThoughtEvent):
            print(f"[THOUGHT] {event.content[:50]}...")
        elif isinstance(event, ActEvent):
            print(f"[ACT] Calling {event.tool_name}({event.tool_input})")
        elif isinstance(event, ObserveEvent):
            result_preview = str(event.result)[:50]
            print(f"[OBSERVE] {event.tool_name}: {result_preview}... ({event.duration_ms}ms)")
        elif isinstance(event, TextDeltaEvent):
            print(f"[TEXT] {event.delta}", end="")
        elif isinstance(event, TextEndEvent):
            print()  # newline after streaming
        elif isinstance(event, CompleteEvent):
            print(f"[COMPLETE] Cost: ${event.cost}")
    
    return events

events = await run_agent()

## 5. 事件分析

In [ ]:
# 分析事件流
print("\n" + "=" * 60)
print("Event Flow Analysis")
print("=" * 60)

from collections import Counter

# 统计事件类型
event_types = Counter(e.event_type.value for e in events)
print("\nEvent counts:")
for event_type, count in event_types.items():
    print(f"  {event_type}: {count}")

# 按类别分组
print("\nEvents by category:")
categories = Counter(get_event_category(e.event_type).value for e in events)
for category, count in categories.items():
    print(f"  {category}: {count}")

In [ ]:
# 完整事件时间线
print("\n" + "=" * 60)
print("Event Timeline (JSON)")
print("=" * 60)

for i, event in enumerate(events):
    print(f"\n{i+1}. {event.event_type.value}:")
    print(json.dumps(event.to_dict(), indent=2, ensure_ascii=False)[:500])

## 6. 导出工具定义为 LLM 格式

In [ ]:
# 导出为 OpenAI 格式（用于实际 LLM 调用）
openai_tools = [tool.to_openai_format() for tool in tools.values()]

print("OpenAI Tools Format:")
print(json.dumps(openai_tools, indent=2, ensure_ascii=False))

## 7. 状态转换追踪

In [ ]:
# 追踪状态转换
print("\n" + "=" * 60)
print("Final Agent State")
print("=" * 60)

print(f"State: {agent.state.value}")
print(f"Steps executed: {agent._step}")

## 8. 模拟 SSE 流

In [ ]:
def format_as_sse(events: List[AgentEvent]) -> str:
    """将事件列表格式化为 SSE 格式。"""
    lines = []
    for event in events:
        data = json.dumps(event.to_dict(), ensure_ascii=False)
        lines.append(f"event: {event.event_type.value}")
        lines.append(f"data: {data}")
        lines.append("")  # 空行分隔
    return "\n".join(lines)


# 显示前几个事件的 SSE 格式
print("SSE Stream Format (first 3 events):")
print("=" * 60)
print(format_as_sse(events[:3]))

## 9. 性能统计

In [ ]:
# 收集性能指标
tool_durations = []
for event in events:
    if isinstance(event, ObserveEvent) and event.duration_ms:
        tool_durations.append((event.tool_name, event.duration_ms))

print("Tool Execution Times:")
print("=" * 40)
for tool_name, duration in tool_durations:
    print(f"  {tool_name}: {duration}ms")

if tool_durations:
    total = sum(d for _, d in tool_durations)
    print(f"\nTotal tool time: {total}ms")
    print(f"Average per tool: {total / len(tool_durations):.1f}ms")

## 总结

本 notebook 演示了 memstack-agent 框架的完整使用流程：

1. **工具定义** - 使用 `@function_to_tool` 创建工具
2. **上下文配置** - 创建 AgentContext 和 ProcessorConfig
3. **事件流** - Agent 通过事件流输出执行过程
4. **状态管理** - 处理器状态机追踪执行阶段
5. **格式转换** - 导出为 OpenAI/Anthropic 格式
6. **SSE 流** - 事件可格式化为 SSE 用于 Web 传输

### 四层架构回顾

```
┌─────────────────────────────────────────┐
│  L4: Agent (ReAct 推理循环)              │
│      - Think → Act → Observe            │
│      - 状态管理、事件发射                 │
├─────────────────────────────────────────┤
│  L3: SubAgent (专业化代理)               │
│      - 路由到专门的子代理                 │
├─────────────────────────────────────────┤
│  L2: Skill (声明式工具组合)              │
│      - 多工具编排                        │
├─────────────────────────────────────────┤
│  L1: Tool (原子能力)                     │
│      - search, read_file, calculate     │
└─────────────────────────────────────────┘
```

### 下一步

- 查看 `docs/memstack-agent-framework-design.md` 了解完整设计
- 查看 `src/infrastructure/agent/` 了解生产实现
- 查看 `src/memstack_agent/tests/` 了解测试模式